For debugging

In [ ]:
import org.apache.spark.AccumulatorParam

object StringAccumulatorParam extends AccumulatorParam[String] {

    def zero(initialValue: String): String = {
        ""
    }

    def addInPlace(s1: String, s2: String): String = {
        s"$s1 $s2"
    }
}

var stringAccum = sc.accumulator("")(StringAccumulatorParam)

Importing libriaries

In [105]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

The class Message represents the message sent between two nodes.
- *source* is the id of the node that sends the message
- *value* is a tuple that contains the values of the message, in the first position the value of the message when the variable is equal to zero and in the second position the value of the message when the variable is equal to one
- *destPosInDistribution* represents the position of the variable node of the message in the adjacent list of the factor node. This is required to find a realization of the variables in the serialized representation of the potential.

The class Node represents all the scope of vertex, except its id.
- *typeOfNode* is 0 if it is a variable node and 1 if it is a factor node
- *neighbors* is the number of the neighbors
- *distribution* is the serialized representation of the potential of a factor node
- *messages* is the list of the messagges collected by the node
- *firstDestination* keep memory of the first node to which a message is already been sent.

In [106]:
case class Message(source: Long, value: (Double, Double), destPosInDistribution: Int)
case class Node(typeOfNode: Int, neighbors: Int, distribution: Array[Double], messages: List[Message], firstDestination: Long)

defined class Message
defined class Node


In [ ]:
Management of the input reading.

In [200]:
val fileName = "../data/input.txt"
var lines = sc.textFile(fileName).collect()
var variablesNumber: Long = 0
var factorsNumber: Long = 0
variablesNumber = lines(0).split(" ")(0).toLong
factorsNumber = lines(0).split(" ")(1).toLong
var tempNodes: Array[(Long, Node)] = new Array[(Long, Node)]((variablesNumber+factorsNumber).toInt) // (id, (type, leaf, distribution))
for (a <- 0 to (variablesNumber+factorsNumber - 1).toInt){
    tempNodes(a) = (a.toLong, Node(0,0, null, List[Message](), -1))
}
var tempEdges: Array[Edge[(Int)]] = new Array[Edge[(Int)]](((variablesNumber+factorsNumber-1)*2).toInt)
var cont = 0
for (a <- 1 to lines.size-1){
    if(a%2 == 1){
        val factorNode = variablesNumber + ((a-1)/2).toLong
        //Pos is the position of the variableNode in the adjacent list of the factorNode
        var pos = 0
        for (variableNode <- lines(a).split(" ")){
            tempEdges(cont) = Edge(variableNode.toLong, factorNode, pos)
            cont = cont + 1
            tempEdges(cont) = Edge(factorNode, variableNode.toLong, pos)
            cont = cont + 1
            pos = pos + 1
        } 
    }else{
        val distribution:  Array[Double] = lines(a).split(" ").map(_.toDouble)
        val factorNode = (variablesNumber + ((a-1)/2).toLong).toInt
        tempNodes(factorNode) = (factorNode, Node(1, 0, distribution, List[Message](), -1))
    }
}

fileName = ../data/input.txt
lines = Array(5 5, 0 1, 0.25 0.25 0.25 0.25, 1 2 3, 0.16706443914081145 0.01909307875894988 0.18138424821002386 0.1431980906921241 0.02386634844868735 0.15513126491646778 0.07159904534606205 0.2386634844868735, 2, 0.4 0.6, 3 4, 0.20 0.30 0.40 0.10, 3, 0.25 0.75)
variablesNumber = 5
factorsNumber = 5
variablesNumber = 5
factorsNumber = 5
tempNodes = Array((0,Node(0,0,null,List(),-1)), (1,Node(0,0,null,List(),-1)), (2,Node(0,0,null,List(),-1)), (3,Node(0,0,null,List(),-1)), (4,Node(0,0,null,List(),-1)), (5,Node(1,0,[D@43eeb8ed,List(),-1)), (6,Node(1,0,[D@6e1c0cb3,List(),-1)), (7,Node(1,0,[D@e69f842,List(),-1)), (8,Node(1,0,[D@1588c85b,List(),-1)), (9,Node(1,0,[D@231733d7,List(),-1)))


tempEdges...


Array((0,Node(0,0,null,List(),-1)), (1,Node(0,0,null,List(),-1)), (2,Node(0,0,null,List(),-1)), (3,Node(0,0,null,List(),-1)), (4,Node(0,0,null,List(),-1)), (5,Node(1,0,[D@43eeb8ed,List(),-1)), (6,Node(1,0,[D@6e1c0cb3,List(),-1)), (7,Node(1,0,[D@e69f842,List(),-1)), (8,Node(1,0,[D@1588c85b,List(),-1)), (9,Node(1,0,[D@231733d7,List(),-1)))

Build the graph.

In [108]:
val nodes: RDD[(VertexId, Node)] = sc.parallelize(tempNodes)
val edges: RDD[Edge[Int]] = sc.parallelize(tempEdges)
val graph = Graph(nodes, edges)

nodes = ParallelCollectionRDD[557] at parallelize at <console>:57
edges = ParallelCollectionRDD[558] at parallelize at <console>:58
graph = org.apache.spark.graphx.impl.GraphImpl@6cec07cb


org.apache.spark.graphx.impl.GraphImpl@6cec07cb

Here the nodes of the graph are joined with a list of the inDegrees of the nodes in order to save the number of neighbors in the node.

In [109]:
val inDegrees: VertexRDD[Int] = graph.inDegrees
//leaves.collect().foreach(println)
val graphWithLeaves = graph.joinVertices(inDegrees){
 case (id, node, neighbors) =>  Node(node.typeOfNode, neighbors, node.distribution, node.messages, node.firstDestination)
}
//graphWithLeaves.vertices.collect().foreach(println)

inDegrees = VertexRDDImpl[574] at RDD at VertexRDD.scala:57
graphWithLeaves = org.apache.spark.graphx.impl.GraphImpl@13c80eb8


org.apache.spark.graphx.impl.GraphImpl@13c80eb8

Given a potential and a list of realization of the variables it returns the value of the potential for this realization.

In [110]:
def getValueFromDistribution(distribution: Array[Double], pos: Array[Int]): Double = {
    var index: Int = 0
    var cont: Int = 1
    for(i <- pos){
        index = index + cont*i
        cont = cont * 2
    }
    distribution(index)
}

getValueFromDistribution: (distribution: Array[Double], pos: Array[Int])Double


Given a variable node it returns the product of all the messagges collected until now.

In [111]:
def computeProductOfMessagesForVariableNode(sourceNode: Node): (Double, Double) = {
    var product: (Double, Double) = (1,1)
    for(msg <- sourceNode.messages){
        product = (product._1*msg.value._1, product._2*msg.value._2)
    }
    product
}

computeProductOfMessagesForVariableNode: (sourceNode: Node)(Double, Double)


Given a list of messages and node it checks if this node is the source of any of the messages.

In [112]:
def checkDestination(messages: List[Message], destinationId: Long): Boolean = {
    var ctrl = true
    for(msg <- messages){
        if(msg.source == destinationId)
            ctrl = false
    }
    ctrl
}

checkDestination: (messages: List[Message], destinationId: Long)Boolean


It computes a message to send to a factor node by considering the special case where it is the first time that this node is ready to send a message to a neighbor.

In [161]:
def computeMessageToFactor(sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int): Message = {
    if(sourceNode.messages.size == sourceNode.neighbors - 1){
        if(checkDestination(sourceNode.messages, destinationId))
            Message(sourceId, computeProductOfMessagesForVariableNode(sourceNode), destPosInDistribution)
        else
            null
    }else if(sourceNode.messages.size == sourceNode.neighbors){
        val product: (Double, Double) = computeProductOfMessagesForVariableNode(sourceNode)
        if(destinationId == sourceNode.firstDestination)
            null
        else{
            var result: Message = null
            for(msg <- sourceNode.messages){
                if(msg.source == destinationId){
                    result = Message(sourceId, (product._1/msg.value._1, product._2/msg.value._2), destPosInDistribution)
                }
            }
            result
        }
    }else{
        null
    }
}

computeMessageToFactor: (sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int)Message


This is the product term inside the computation of a message destinated to a variable node.

In [162]:
def computeProductOfMessagesForFactorNode(factorNode: Node, destinationId: Long, states: Array[Int]): Double = {
    var product: Double = 1
    for(msg <- factorNode.messages){
        if(msg.source != destinationId){
            if(states(msg.destPosInDistribution) == 0)
                product = product * msg.value._1
            else
                product = product * msg.value._2
        }
    }
    product
}

computeProductOfMessagesForFactorNode: (factorNode: Node, destinationId: Long, states: Array[Int])Double


This is the summation term inside the computation of a message destinated to a variable node.

In [163]:
def computeMessageToVariableSummation(pos: Int, factorNode: Node, destinationId: Long, states: Array[Int], destPosInDistribution: Int): Double = {
    if(pos == factorNode.neighbors)
        getValueFromDistribution(factorNode.distribution, states)*computeProductOfMessagesForFactorNode(factorNode, destinationId, states)
    else{
        if(pos == destPosInDistribution)
            computeMessageToVariableSummation(pos+1, factorNode, destinationId, states, destPosInDistribution)
        else{
            var temp: Double = 0
            states(pos) = 0
            temp = temp + computeMessageToVariableSummation(pos+1, factorNode, destinationId, states, destPosInDistribution)
            states(pos) = 1
            temp = temp + computeMessageToVariableSummation(pos+1, factorNode, destinationId, states, destPosInDistribution)
            states(pos) = -1
            temp
        }
    }
}

computeMessageToVariableSummation: (pos: Int, factorNode: Node, destinationId: Long, states: Array[Int], destPosInDistribution: Int)Double


It computes a message for a variable node for both  the realizations of the variable.

In [164]:
def computeMessageToVariable2(sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int): (Double, Double) = {
    var result: (Double, Double) = (0,0)
    var states = Array.fill[Int](sourceNode.neighbors)(-1)
    states(destPosInDistribution) = 0
    result = (computeMessageToVariableSummation(0, sourceNode, destinationId, states, destPosInDistribution), 0)
    states(destPosInDistribution) = 1
    result = (result._1, computeMessageToVariableSummation(0, sourceNode, destinationId, states, destPosInDistribution))
    result
}
    


computeMessageToVariable2: (sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int)(Double, Double)


It computes a message to send to a variable node by considering the special case where it is the first time that this node is ready to send a message to a neighbor.

In [165]:
def computeMessageToVariable(sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int): Message = {
    if(sourceNode.messages.size == sourceNode.neighbors - 1){
        if(checkDestination(sourceNode.messages, destinationId))
            Message(sourceId, computeMessageToVariable2(sourceId, sourceNode, destinationId, destPosInDistribution), -1)
        else
            null
    }else if(sourceNode.messages.size == sourceNode.neighbors){
        if(destinationId == sourceNode.firstDestination)
            null
        else{
            //stringAccum += sourceId + " " + destinationId + "\n"
            Message(sourceId, computeMessageToVariable2(sourceId, sourceNode, destinationId, destPosInDistribution), -1)
        }
    }else{
        null
    }
}

computeMessageToVariable: (sourceId: Long, sourceNode: Node, destinationId: Long, destPosInDistribution: Int)Message


It combines two incoming messages in the same list.

In [166]:
def mergeMsg(msg1: List[Message], msg2: List[Message]): List[Message] = {
    msg1:::msg2
}

mergeMsg: (msg1: List[Message], msg2: List[Message])List[Message]


In [185]:
def vprog(vertexId: VertexId, value: Node, message: List[Message]): Node = {
    if(message.head.source == -1){ //when receive initial message the node doesn't change
        Node(value.typeOfNode, value.neighbors, value.distribution, value.messages, value.firstDestination)
    }
    else{ 
        val newMessages = value.messages:::message
        var firstDestination: Long = value.firstDestination
        if(newMessages.size == value.neighbors && message.size == 1)
            firstDestination = message.head.source
        Node(value.typeOfNode, value.neighbors, value.distribution, newMessages, firstDestination)
    }
}

vprog: (vertexId: org.apache.spark.graphx.VertexId, value: Node, message: List[Message])Node


In [186]:
//tofix
def sendMsg(triplet: EdgeTriplet[Node, Int]): Iterator[(VertexId, List[Message])] = {
    val sourceId = triplet.srcId
    val sourceNode = triplet.srcAttr
    val destinationId = triplet.dstId
    if(sourceNode.messages.size == 0){
        if(sourceNode.neighbors == 1){ //send message from the leaf
            if(sourceNode.typeOfNode == 0)
                Iterator((destinationId, List(Message(sourceId, (1,1), triplet.attr))))
            else
                Iterator((destinationId, List(Message(sourceId, (getValueFromDistribution(sourceNode.distribution, Array(0)), getValueFromDistribution(sourceNode.distribution, Array(1))), -1))))
        }else{
            Iterator.empty
        }
    }else{
        if(sourceNode.neighbors == 1){ //a leaf should not send any message after the first superstep
            Iterator.empty
        }else{
            if(sourceNode.typeOfNode == 0){
                val msg = computeMessageToFactor(sourceId, sourceNode, destinationId, triplet.attr)
                //stringAccum += sourceId + " " + destinationId + " " + msg + "\n"
                if(msg == null) // GraphX examine all the adjacent edges to send messages so it is required to check if the edge should receive or not the message
                    Iterator.empty
                else
                    Iterator((destinationId, List(msg)))
            }
            else{
                val msg = computeMessageToVariable(sourceId, sourceNode, destinationId, triplet.attr)
                //stringAccum += sourceId + " " + destinationId + " " + msg + "\n"
                if(msg == null)
                    Iterator.empty
                else
                    Iterator((destinationId, List(msg)))
            }
        }   
    } 
}

sendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[Node,Int])Iterator[(org.apache.spark.graphx.VertexId, List[Message])]


In [187]:
stringAccum = sc.accumulator("")(StringAccumulatorParam)

stringAccum = 


In [188]:
val initialMsg = List(Message(-1, (-1, -1), -1))
val maxIterations = ((variablesNumber+factorsNumber)*2).toInt*400000
val finalGraph = graphWithLeaves.pregel(initialMsg, maxIterations, EdgeDirection.Out)(vprog, sendMsg, mergeMsg)

initialMsg = List(Message(-1,(-1.0,-1.0),-1))
maxIterations = 8000000
finalGraph = org.apache.spark.graphx.impl.GraphImpl@284cb28a


org.apache.spark.graphx.impl.GraphImpl@284cb28a

In [189]:
val temp = finalGraph.vertices.map(x => (x._1, x._2.messages.size, x._2.neighbors))
temp.collect().foreach(println)

(4,1,1)
(0,1,1)
(8,2,2)
(1,2,2)
(9,1,1)
(5,2,2)
(6,3,3)
(2,2,2)
(3,3,3)
(7,1,1)


temp = MapPartitionsRDD[1344] at map at <console>:51


MapPartitionsRDD[1344] at map at <console>:51

In [190]:
println(stringAccum)

                          3 6 null
 2 6 Message(2,(0.4,0.6),1)
  3 8 null
 2 7 null
  3 9 null
 8 3 Message(8,(0.6000000000000001,0.4),-1)
 8 4 null
  5 0 null
 5 1 Message(5,(0.5,0.5),-1)
              1 5 null
  3 9 null
  6 3 null
 3 8 null
  1 6 Message(1,(0.5,0.5),0)
 6 1 null
 6 2 null
 3 6 Message(3,(0.15000000000000002,0.30000000000000004),2)
                6 3 Message(6,(0.13460620525059663,0.12887828162291168),-1)
  6 1 Message(6,(0.042100238663484495,0.07560859188544153),-1)
 6 2 Message(6,(0.04081145584725537,0.07088305489260144),-1)
              1 5 Message(1,(0.042100238663484495,0.07560859188544153),1)
  3 9 Message(3,(0.08076372315035799,0.05155131264916468),0)
  1 6 null
 3 6 null
 2 6 null
  3 8 Message(3,(0.03365155131264916,0.09665871121718378),0)
 2 7 Message(2,(0.04081145584725537,0.07088305489260144),0)
               5 0 Message(5,(0.029427207637231508,0.029427207637231508),-1)
 5 1 null
   8 3 null
 8 4 Message(8,(0.03572792362768497,0.02312649164677804),-1)


In [199]:
def newTuple(tuple: (Double, Double)): (Double, Double, Double) = {
    (tuple._1/(tuple._1+tuple._2), tuple._2/(tuple._1+tuple._2), tuple._1+tuple._2)
}
finalGraph.vertices.filter(x => x._2.typeOfNode == 0).map(x => (x._1, newTuple(computeProductOfMessagesForVariableNode(x._2)))).sortByKey().collect().foreach(println)

newTuple: (tuple: (Double, Double))(Double, Double, Double)


(0,(0.5,0.5,0.058854415274463015))
(1,(0.35766423357664234,0.6423357664233575,0.058854415274463015))
(2,(0.2773722627737226,0.7226277372262773,0.058854415274463015))
(3,(0.3430656934306569,0.6569343065693432,0.05885441527446301))
(4,(0.6070559610705597,0.39294403892944035,0.05885441527446301))


In [192]:
//finalGraph.vertices.map(x => (x._1, x._2.firstDestination)).collect().foreach(println)

(4,8)
(0,5)
(8,3)
(1,6)
(9,3)
(5,1)
(6,-1)
(2,6)
(3,6)
(7,2)
